<a href="https://colab.research.google.com/github/IFuentesSR/AET_moderate_eval/blob/main/rain_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='zzzz')

In [ ]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
gpm = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06")
years = ee.List.sequence(1981, 2022)
basins = ee.FeatureCollection('xxxxxxxxxxxxxxxx')

In [ ]:
def daily2Monthly(y):
    def inner(m):
        date = ee.Date.fromYMD(y, m, 1)
        date1 = date.advance(1, 'month')
        subset = chirps.filterDate(date, date1)
        return subset.reduce('sum').setMulti({'system:time_start':date.millis(),
                                              'date':date.format('YYYY-MM-dd'),
                                              'size':subset.size()})
    months = ee.List.sequence(1, 12).map(inner)
    return months


def rainChirps(collection):
    def wrap(fea):
        def inner(img):
            rain = img.reduceRegion('mean', fea.geometry(), 5000).values().get(0)
            return ee.Feature(None, {'rain':rain,
                                    'Cuenca':fea.get('NOM_CUEN'),
                                    'date':ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')})

        return collection.map(inner)
    return wrap


In [ ]:
rain_chirps = ee.ImageCollection.fromImages(years.map(daily2Monthly).flatten())
rain_chirps = rain_chirps.sort('system:time_start').filter(ee.Filter.gte('size', 28))
chirps_rain = basins.map(rainChirps(rain_chirps)).flatten()
ee.batch.Export.table.toDrive(chirps_rain, 'rain_chirps_xxx').start()

In [ ]:
def rainGPM(collection):
    def wrap(fea):
        def inner(img):
            rain = img.reduceRegion('mean', fea.geometry(), 11132).values().get(0)
            return ee.Feature(None, {'rain':rain,
                                    'Cuenca':fea.get('NOM_CUEN'),
                                    'date':ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')})

        return collection.map(inner)
    return wrap

In [ ]:
gpm_rain = basins.map(rainGPM(gpm)).flatten()
ee.batch.Export.table.toDrive(gpm_rain, 'rain_gpm_xxxx').start()